In [1]:
import time
import pyspark
import numpy as np
from operator import add

In [2]:
workspace='hdfs:/user/bigdata/data/'
file_name='Facebook_Microsoft'
out_byHour = 'file:///home/bigdata/Documents/BD_HW/HW2/output/Byhour'
out_byDay = 'file:///home/bigdata/Documents/BD_HW/HW2/output/Byday'
sc = pyspark.SparkContext(appName='hw2-2')
sqlContext = pyspark.SQLContext(sc)

In [3]:
fileList=['Facebook_Economy', 'Facebook_Microsoft', 'Facebook_Obama', 'Facebook_Palestine',\
          'GooglePlus_Economy', 'GooglePlus_Microsoft', 'GooglePlus_Obama', 'GooglePlus_Palestine',\
          'LinkedIn_Economy', 'LinkedIn_Microsoft', 'LinkedIn_Obama', 'LinkedIn_Palestine']
header_per_day=['IDLink'] + ['TS'+str((count+1)*72) for count in range(2)]
header_per_hour=['IDLink'] + ['TS'+str((count+1)*3) for count in range(48)]
print(header_per_day)
print(header_per_hour)

['IDLink', 'TS72', 'TS144']
['IDLink', 'TS3', 'TS6', 'TS9', 'TS12', 'TS15', 'TS18', 'TS21', 'TS24', 'TS27', 'TS30', 'TS33', 'TS36', 'TS39', 'TS42', 'TS45', 'TS48', 'TS51', 'TS54', 'TS57', 'TS60', 'TS63', 'TS66', 'TS69', 'TS72', 'TS75', 'TS78', 'TS81', 'TS84', 'TS87', 'TS90', 'TS93', 'TS96', 'TS99', 'TS102', 'TS105', 'TS108', 'TS111', 'TS114', 'TS117', 'TS120', 'TS123', 'TS126', 'TS129', 'TS132', 'TS135', 'TS138', 'TS141', 'TS144']


In [4]:
platforms='Facebook'

out_hour=open('output/' + platforms + '_hour.txt','w')
out_day=open('output/' + platforms + '_day.txt','w')
for fileName in fileList:
    social_feedback_data=sqlContext.read.format('csv')\
    .option("header", "true")\
    .option("inferschema", "true")\
    .load(workspace+fileName+'.csv')
    popularityAvg_per_hour=social_feedback_data.select(header_per_hour).rdd.map(list)\
                                               .flatMap(lambda x:((x[0], element) for element in x[1:]))\
                                               .reduceByKey(add).map(lambda x:(x[0], x[1]/48)).sortByKey()\
                                               .map(lambda x:('ID'+str(x[0]), x[1])).collect()
    popularityAvg_per_day=social_feedback_data.select(header_per_day).rdd.map(list)\
                                              .flatMap(lambda x:((x[0], element) for element in x[1:]))\
                                              .reduceByKey(add).map(lambda x:(x[0], x[1]/2)).sortByKey()\
                                              .map(lambda x:('ID'+str(x[0]), x[1])).collect()
    if fileName.split('_')[0] != platforms:
        out_hour.close()
        out_day.close()
        platforms=fileName.split('_')[0]
        out_hour=open('output/' + platforms + '_hour.txt','w')
        out_day=open('output/' + platforms + '_day.txt','w')
    out_hour.write(fileName + '\n')
    out_day.write(fileName + '\n')
    for i in range(len(popularityAvg_per_hour)):
        out_hour.write(str(popularityAvg_per_hour[i]) + '\n')
    out_hour.write('\n')
    for i in range(len(popularityAvg_per_day)):
        out_day.write(str(popularityAvg_per_day[i]) + '\n')
    out_day.write('\n')
out_hour.close()
out_day.close()

In [5]:
sc.stop()